# Agentic RAG

## 核心思想

Agentic RAG 的核心不是使用更复杂的模型，而是**让模型使用 tools 完成任务**。

和一次性把文档塞进 Prompt 就生成答案的 Native RAG 相比，Agentic RAG 让大模型扮演一个**决策-执行的控制器**。

> 模型通过自主决策调用 tools 实现的 RAG 过程，就可以称为 Agentic RAG。无论这个过程是发生在重写 query、检索阶段或者精读阶段，只要有模型的自主决策过程，就可以称为 Agentic RAG。

## 运行流程

```
User Query -> LLM (GPT/Claude/DeepSeek) <-> Tools (MCP Server)
                    |                         ^
              Use Tool ----------------------->
                    <--------------- Tool Result
                    |
         Recycle until model decide response to user
                    |
               Response
```

**核心要点：Search Ability As a Tool**

模型就可以通过可用的工具获取到想要的信息从而给出更合适的回复。

## 与传统 RAG 的对比

| 特性 | 传统 RAG | Agentic RAG |
|------|----------|-------------|
| 流程 | 一次性流水线 | 多轮迭代循环 |
| Query 处理 | 直接使用原始 query | 可改写、分解、扩展 |
| 检索策略 | 固定策略 | 动态选择 |
| 结果评估 | 无 | Agent 评估相关性 |
| 精读能力 | 无 | 可对文档深度理解 |
| 决策主体 | 代码逻辑 | LLM 自主决策 |

## 关键能力

1. **Query 理解与改写**：Agent 分析用户意图，改写或分解复杂问题
2. **多轮检索**：根据初次检索结果决定是否需要补充检索
3. **结果评估**：评估检索结果的相关性，过滤低质量内容
4. **Reason-Act-Observe 循环**：按需多轮调用工具

## 实现方式

本示例采用 **单 Agent + Tools** 模式（非 Subagent 模式）：
- 1 个 LLM 作为大脑
- 多个 Tools 封装 RAG 能力
- Agent 自主决策调用哪个工具

In [22]:
# 环境初始化
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

qwen_plus_model = init_chat_model(
    "qwen-plus",
    model_provider="openai",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    temperature=0.5
)

#向量化
from langchain_community.embeddings import DashScopeEmbeddings

embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.load_local(
    "./faiss_index",
    embeddings,
    allow_dangerous_deserialization=True  
)


In [23]:
from pydantic import BaseModel, Field
from langchain.tools import tool

class expand_and_keyword_format(BaseModel):
    """对用户的query进行改写和提取关键词的结果"""
    expand_query:str = Field(...,description =  "对query进行扩写/改写的结果，可以不改写")
    keyword: str = Field(...,description = "在query中提取一个最重要的简短关键字")

@tool
def expand_and_keyword(query):
    """
        对用户输入的query进行改写和提取一个最重要的关键字
        :param query:str, 必填 -是用户的输入，在此基础上进行改写和提取关键字。
        :return expand_query: str, 必填- 对用户query进行扩写/改写的结果，可以不改写
        :return keyword：str,对用户的输入提取一个最重要的关键字。
    """

    expand_and_keyword_conversation = [
        {"role":"system","content":"你是一个乐于助人的助手，负责给用户的query进行改写/扩写，同时，在query中提取一个最重要的关键词"},
        {"role":"user","content":query},
    ]
    #格式化输出这样更简单，不需要用agent的能力
    model_with_structure = model.with_structure_output(expand_and_keyword_format)
    response = model_with_structure.invoke(expand_and_keyword_conversation)
    expand_query = response.expand_query
    keyword = response.keyword
    return expand_query,keyword

In [24]:
@tool
def retrieval_augment(query,keyword):
    """
        【rag检索增强】基于用户输入的查询语句和核心关键词，从知识库中检索相关文档并增强结果。
        核心逻辑：先按query检索top10文档，若有keyword则调整匹配得分（含关键词的文档得分-0.1）,按照得分升序排序，拼接前5条文档内容返回。
        :param query:str, 必填，用户输入的自然语言查询语句，作为基础检索条件。
        :param keyword：str,必填，从用户输入中提取的核心关键词，用于优化检索结果权重。
        :return related_doc: str, 按优化后得分排序库文档内容的检索结果，以双换行符分割拼接后的字符串 
    """
    # 用 with_score 获取分数
    docs = vectorstore.similarity_search_with_score(query,k=10)

     # 如果有关键词，调整得分
    if keyword:
        adjusted = []
        for doc, score in docs_with_scores:
            if keyword in doc.page_content:
                score -= 0.1  # 含关键词的得分更低（FAISS分数越低越相似）
            adjusted.append((doc, score))
        docs_with_scores = adjusted

     
    # 按分数升序排序（越小越相似）
    docs_with_scores = sorted(docs_with_scores, key=lambda x: x[1])

    top_docs = docs_with_scores[:5]
    related_doc = '\n\n'.join([doc.page_content for doc, score in top_docs])
    
    return related_doc

In [25]:
class summary_related_doc_format(BaseModel):
    '''
        对用户的query相关的片段进行筛选
    '''
    summary_related_doc_res:str = Field(...,description = '对用户query相关的片段进行筛选')
@tool
def summary_related_doc(query,related_doc):
    '''
        片段精读。根据用户的query,在related_doc原始片段中把与query相关的片段进行摘取，不要进行改写，仅把相关部分进行筛选复制。

        :param query:str ,必填，用户输入的自然语言查询语句，作为基础的检索条件
        :param related_doc:str,必填，原始的文档片段
        :return :summary_related_doc_res,筛选后的片段
    '''

    query_and_related_doc = f'这是用户的query:{query}\n\n这是原文的众多片段:{related_doc}'
    summary_related_doc_conversation = [
        {"role":"system","content":"你是一个乐于助人的助手，现在又很多原文片段，你需要对用户query相关的片段进行筛选，不要改动原文，只要把相关的片段进行复制筛选即可"},
        {"role":"user","content":query_and_related_doc}
    ]

    model_with_structure = model.with_structure_output(summary_related_doc_format)
    response = model_with_structure.invoke(summary_related_doc_conversation)
    summary_related_doc_res = response.summary_related_doc_res
    return summary_related_doc_res

In [26]:
system_prompt = '''
你是一名助人为乐的助手，你会根据用户的问题进行判断：

- 如果用户问题与校规不相关，直接回答，不要调用工具。

- 如果用户问题与校规相关，按以下步骤操作：
  1. 调用 expand_and_keyword 改写问题、提取关键词
  2. 调用 retrieval_augment 检索相关文档
  3. 若检索结果过长，调用 summary_related_doc 精读筛选
  4. 根据检索内容回答用户问题
  5. 若未找到答案，可重试一次（最多两次）
  6. 两次后仍无答案，回复：没有在文档中检索到相关内容

回答时请基于检索到的文档内容，不要编造。
'''

In [ ]:
tools = [expand_and_keyword, retrieval_augment, summary_related_doc]

agent = create_agent(
    model=qwen_plus_model,
    tools=tools,
    system_prompt=system_prompt,
    middleware=[
        # 全局限制：所有 tool 总共最多调用 10 次
        ToolCallLimitMiddleware(run_limit=10),
        
        # 单个 tool 限制：每个 tool 最多调用 2 次
        ToolCallLimitMiddleware(tool_name="expand_and_keyword", run_limit=2),
        ToolCallLimitMiddleware(tool_name="retrieval_augment", run_limit=2),
        ToolCallLimitMiddleware(tool_name="summary_related_doc", run_limit=2),
    ]

)

# 调用
response = agent.invoke({"messages": [{"role": "user", "content": "学生可以转专业吗"}]})


NameError: name 'model' is not defined

In [ ]:
# 测试不相关问题
query = '你是谁？'
response = agent.invoke({"messages": [{"role": "user", "content": query}]})

response



In [ ]:
#测试相关的问题
query = '两周未参加学校活动会被退学吗'

response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]}
)

# 遍历所有消息，查看 Agent 的执行轨迹
for res in response['messages']:
    print(res.name, end='\n\n')


In [ ]:
# 测试不相关问题
query = '如果想要换班级怎么走流程'
response = agent.invoke({"messages": [{"role": "user", "content": query}]})
response